# Manage EC2 via CLI



Credits **Educative.io**

Install AWS CLI:
```zsh
curl "https://awscli.amazonaws.com/AWSCLIV2.pkg" -o "AWSCLIV2.pkg"
sudo installer -pkg AWSCLIV2.pkg -target /
which aws
aws --version
```

Check out help:

```zsh
aws ec2 help  | grep <some word>
```

To create a user (and also make sure to set up the AmazonEC2FullAccess user policy for the IAM user):

```zsh
aws iam create-user --user-name testuser
aws iam create-group --group-name testgroup
aws iam attach-group-policy --group-name testgroup --policy-arn arn:aws:iam::aws:policy/AdministratorAccess
aws iam attach-group-policy --group-name testgroup --policy-arn arn:aws:iam::aws:policy/AmazonEC2FullAccess
aws iam add-user-to-group --user-name testuser --group-name testgroup
```



To see key value pairs:
```zsh
aws ec2 describe-key-pairs
aws ec2 describe-key-pairs --output table

aws ec2 create-key-pair --key-name ec2project --query 'KeyMaterial' --output text > ec2project.pem
```

A VPC (virtual private cloud) is a virtual private network in Amazon’s data centers that has restrictions on its access. Within this VPC, all your instances and services can communicate, but other AWS customers can’t see them.

```zsh
aws ec2 describe-vpcs
```

In Amazon EC2, security groups are the virtual firewalls that control the inbound and outbound traffic for EC2 instances. 

```zsh
aws ec2 describe-security-groups --filters Name=vpc-id,Values=$VPC_ID --query "SecurityGroups[*].GroupId"
aws ec2 describe-security-groups --filters Name=vpc-id,Values=$VPC_ID --query "SecurityGroups[*].GroupId" --output table
```



Amazon Subnets are smaller ranges of IP addresses within a VPC:

```zsh
aws ec2 describe-availability-zones --query "AvailabilityZones[*].{"RegionName":RegionName,"ZoneName":ZoneName}" --output table
```

```zsh
aws ec2 create-subnet --vpc-id $VPC_ID --cidr-block 172.31.0.0/16 --availability-zone us-west-1c --tag-specifications 'ResourceType=subnet, Tags=[{Key=name,Value=subnet-public-a},{Key=learning, Value=educative}]'
```

```zsh
aws ec2 describe-subnets
aws ec2 modify-subnet-attribute --subnet-id $SUBNET_ID --map-public-ip-on-launch
```

The default security group has some default inbound and outbound rules.
Let's add port 5000 utilized by Flask
```zsh
aws ec2 authorize-security-group-ingress --group-id $SG_ID --protocol tcp --port 5000 --cidr 0.0.0.0/0
```
and port 22 used by ssh:
```zsh
aws ec2 authorize-security-group-ingress --group-id $SG_ID --protocol tcp --port 22 --cidr 0.0.0.0/0
```


To create an instance first find the AMI ID of Linux you want to use (for Ubuntu: `ami-0ce2cb35386fc22e9` for Amazon Linux: Amazon Linux 2 AMI:
`ami-0082110c417e4726e`)
```zsh
aws ec2 run-instances --image-id $AMI_ID --count 1 --instance-type t2.micro --key-name ec2project --security-group-id $SG_ID --subnet-id $SUBNET_ID
``` 

To check the status of instances:
```zsh
aws ec2 describe-instances
aws ec2 describe-instances --instance-id i-04918bcdbf7cffd64 --output table
aws ec2 describe-instances --instance-ids i-04918bcdbf7cffd64 --output table --query 'Reservations[*].Instances[*].[InstanceId,State.Name]'
aws ec2 describe-instances --output table --query 'Reservations[*].Instances[*].[InstanceId,State.Name]'
```

To ssh into a EC2 instance:
```
ssh -i ec2project.pem ec2-user@13.57.29.210
```

To deploy anything to EC2 instance first change mode:
`chmod 400 ec2project.pem`

```zsh
scp -i ec2project.pem flask.tar.gz ec2-user@13.57.29.210:flask.tar.gz
```

For Amazon Linux AMI username is `ec2-user`.

Install pip and packages:
```zsh
curl -O https://bootstrap.pypa.io/get-pip.py
sudo python3 get-pip.py
python3 -m pip install flask flask_sqlalchemy
```

```zsh
tar -xvf flask.tar.gz
export FLASK_APP=application.py 
flask run --host=0.0.0.0
```

To terminate instance:
```zsh
aws ec2 terminate-instances --instance-ids i-0ab286b95c7dd99da
```

To delete key-pair:
```zsh
aws ec2 delete-key-pair --key-name ec2project
```
To delete subnet:
```zsh
aws ec2 delete-subnet --subnet-id $SUBNET_ID
```